In [11]:
class PicoMotor:
    
    _MIN_SPEED = 50
    _MAX_SPEED = 100
    _MIN_DIFF = 10
    
    def __init__(self, serial_port):
        self.serial_port = serial_port
        
    def _send(self, left, right):
        cmd = f'MOTOR:left:{left}:right:{right}'.encode()
        serial_port.write(cmd)
        
    def _bounding_speed(self, speed):        
        if abs(speed) < self._MIN_SPEED:
            print(f'[PicoMotor]: Lowest speed is {self._MIN_SPEED}, reset value {speed} to it.')
            return self._MIN_SPEED if speed >= 0 else -self._MIN_SPEED
        
        if abs(speed) > self._MAX_SPEED:
            print(f'[PicoMotor]: Max speed is {self._MAX_SPEED}, reset value {speed} to it.')
            return self._MAX_SPEED if speed >= 0 else -self._MAX_SPEED
        
        return speed
    
    def _get_diff_speed(self, speed, diff):
        if speed >= 0:
            return speed - diff
        
        return speed + diff
    
    def _is_need_change_to_rotate(self, speed, diff_speed):
#         return abs(speed) == self._MIN_SPEED or (abs(speed) - abs(diff_speed)) < self._MIN_DIFF
        return (
            speed == diff_speed or
            ((abs(speed) - abs(diff_speed)) < self._MIN_DIFF and abs(diff_speed) == self._MIN_SPEED)
        )
    
    def go(self, speed, diff):
        '''
        high level，會依照 speed, diff 的值來決定要 直走 還是 轉彎
        speed:
            + : forward
            - : backward
        diff:
            + : right
            - : left
        '''
        if speed == 0:
            return self.stop()
            
        if diff == 0:
            return self.go_straight(speed)
        
        return self.go_turn(speed, diff)
        
    def go_straight(self, speed):
        '''
        Always 直走
        speed:
            + : forward
            - : backward
        '''
        print(f'[PicoMotor]: go_straight, speed={speed}')
        speed = self._bounding_speed(speed)
        self._send(speed, speed)
        
    def go_turn(self, speed, diff):
        '''
        轉彎專用
        speed:
            + : forward
            - : backward
        diff:
            + : right
            - : left
        '''
        print(f'[PicoMotor]: go_turn, speed={speed}, diff={diff}')
        if speed == 0:
            return
        
        speed = self._bounding_speed(speed)
        diff_speed = self._bounding_speed(self._get_diff_speed(speed, abs(diff)))
        print(speed, diff_speed)
        
        if self._is_need_change_to_rotate(speed, diff_speed):
            if (speed > 0 and diff >= 0) or (speed < 0 and diff < 0):
                print('[PicoMotor]: change to right_rotate')
                self.right_rotate()
            else:
                print('[PicoMotor]: change to left_rotate')
                self.left_rotate()
        else:    
            if diff >= 0:
                self._send(speed, diff_speed)
            else:
                self._send(diff_speed, speed)

    def turn_left(self, speed, diff):
        '''
        明確左轉使用
        speed: 
            + : forward
            - : backward
        diff:
            always +
        '''
        print(f'[PicoMotor]: turn_left, speed={speed}, diff={diff}')
        return self.go_turn(speed, -diff)

    def turn_right(self, speed, diff):
        '''
        明確右轉使用
        speed: 
            + : forward
            - : backward
        diff:
            always +
        '''
        print(f'[PicoMotor]: turn_tight, speed={speed}, diff={diff}')
        return self.go_turn(speed, diff)

    def left_rotate(self):
        '''
        原地向左旋轉
        '''
        print(f'[PicoMotor]: left_rotate')
        self._send(-self._MIN_SPEED, self._MIN_SPEED)
        
    def right_rotate(self):
        '''
        原地向右旋轉
        '''
        print(f'[PicoMotor]: right_rotate')     
        self._send(self._MIN_SPEED, -self._MIN_SPEED)
        
    def stop(self):
        self._send(0, 0)
    
    

In [2]:
import smbus

I2C_CH = 0

MOTOR_ADDR = 0x50
CMD_DOMAIN_MOTOR = 0x01

class I2CHelper:
    
    def __init__(self, channel = I2C_CH):
        self.i2c = smbus.SMBus(channel)
        
    def write(self, cmd):
        cmd = cmd.decode()
        if cmd.startswith('MOTOR'):
            self._write_motor_cmd(cmd)
        
        else:
            print('This cmd is not support.', cmd)
        
    def _write_motor_cmd(self, cmd):
        '''
        str cmd: MOTOR:left:{left}:right:{right}
        i2c cmd: [CMD_DOMAIN_MOTOR, LEFT_SIGN, LEFT_SPEED, RIGHT_SIGN, RIGHT_SPEED]
        '''
        data = cmd.split(':')
        lspeed = int(data[2])
        rspeed = int(data[4])
        lsign = int(lspeed >= 0)
        rsign = int(rspeed >= 0)
        self.i2c.write_block_data(
            MOTOR_ADDR,
            0x01,
            [CMD_DOMAIN_MOTOR, lsign, abs(lspeed), rsign, abs(rspeed)]
        )

In [ ]:
import serial

_DEFAULT_PORT = '/dev/ttyTHS1'
_BAUDRATE = 115200

serial_port = serial.Serial(
    port=port,
    baudrate=_BAUDRATE,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
)

In [5]:
class FakeSerial:
    def write(self, cmd):
        print(f'[FakeSerial]: {cmd}')
        
serial_port = FakeSerial()

In [ ]:
m = PicoMotor(serial_port)
print('==== Test forward ====')
m.go_straight(100)
m.go_straight(60)
m.go_straight(20)
print('\n==== Test backward ====')
m.go_straight(-100)
m.go_straight(-60)
m.go_straight(-20)
print('\n==== Test left ====')
m.turn_left(100, 10)
m.turn_left(60, 10)
m.turn_left(60, 20)
m.turn_left(50, 10)
m.turn_left(59, 10)
m.turn_left(30, 10)
print('\n==== Test left ====')
m.go_turn(100, -10)
m.go_turn(60, -10)
m.go_turn(60, -20)
m.go_turn(50, -10)
m.go_turn(59, -10)
m.go_turn(30, -10)
print('\n==== Test right ====')
m.turn_right(100, 10)
m.turn_right(60, 10)
m.turn_right(60, 20)
m.turn_right(50, 10)
m.turn_right(59, 10)
m.turn_right(30, 10)
print('\n==== Test right ====')
m.go_turn(100, 10)
m.go_turn(60, 10)
m.go_turn(60, 20)
m.go_turn(50, 10)
m.go_turn(59, 10)
m.go_turn(30, 10)

In [3]:
import ipywidgets.widgets as widgets
from IPython.display import display

speed_slider = widgets.IntSlider(description='speed', min=-100, max=100, step=1, orientation='vertical')
direct_slider = widgets.IntSlider(description='direct', min=-100, max=100, step=1, orientation='horizontal')

slider_container = widgets.HBox([speed_slider, direct_slider])
display(slider_container)

[PicoMotor]: go_turn, speed=100, diff=-80
[PicoMotor]: Lowest speed is 50, reset value 20 to it.
100 50
[FakeSerial]: b'MOTOR:left:50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-77
[PicoMotor]: Lowest speed is 50, reset value 23 to it.
100 50
[FakeSerial]: b'MOTOR:left:50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-80
[PicoMotor]: Lowest speed is 50, reset value 20 to it.
100 50
[FakeSerial]: b'MOTOR:left:50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-82
[PicoMotor]: Lowest speed is 50, reset value 18 to it.
100 50
[FakeSerial]: b'MOTOR:left:50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-95
[PicoMotor]: Lowest speed is 50, reset value 5 to it.
100 50
[FakeSerial]: b'MOTOR:left:50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 0 to it.
100 -50
[FakeSerial]: b'MOTOR:left:-50:right:100'
[PicoMotor]: go_turn, speed=100, diff=-99
[PicoMotor]: Lowest speed is 50, reset value 1 to it.
100 50
[FakeSerial]: b'MOTOR:left

[PicoMotor]: go_turn, speed=100, diff=-72
[PicoMotor]: Lowest speed is 50, reset value 28 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-72
[PicoMotor]: Lowest speed is 50, reset value 28 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-72
[PicoMotor]: Lowest speed is 50, reset value 28 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-70
[PicoMotor]: Lowest speed is 50, reset value 30 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-70
[PicoMotor]: Lowest speed is 50, reset value 30 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-70
[PicoMotor]: Lowest speed is 50, reset value 30 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-68
[PicoMotor]: Lowest speed is 50, reset value 32 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-68
[PicoMotor]: Lowest speed is 50, reset value 32 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-68
[PicoMotor]: Lowest speed is 50, reset value 32 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-65
[PicoMotor]: Lowest sp

[PicoMotor]: go_turn, speed=100, diff=-20
100 80
[PicoMotor]: go_turn, speed=100, diff=-20
100 80
[PicoMotor]: go_turn, speed=100, diff=-20
100 80
[PicoMotor]: go_turn, speed=100, diff=-25
100 75
[PicoMotor]: go_turn, speed=100, diff=-25
100 75
[PicoMotor]: go_turn, speed=100, diff=-25
100 75
[PicoMotor]: go_turn, speed=100, diff=-38
100 62
[PicoMotor]: go_turn, speed=100, diff=-38
100 62
[PicoMotor]: go_turn, speed=100, diff=-38
100 62
[PicoMotor]: go_turn, speed=100, diff=-56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-72
[PicoMotor]: Lowest speed is 50, reset value 28 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-72
[PicoMotor]: Lowest speed is 50, reset value 28 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=-

[PicoMotor]: go_turn, speed=100, diff=51
[PicoMotor]: Lowest speed is 50, reset value 49 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=51
[PicoMotor]: Lowest speed is 50, reset value 49 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=51
[PicoMotor]: Lowest speed is 50, reset value 49 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=60
[PicoMotor]: Lowest speed is 50, reset value 40 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=60
[PicoMotor]: Lowest speed is 50, reset value 40 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=60
[PicoMotor]: Lowest speed is 50, reset value 40 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=72
[PicoMotor]: Lowest speed is 50,

[PicoMotor]: go_turn, speed=100, diff=58
[PicoMotor]: Lowest speed is 50, reset value 42 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=58
[PicoMotor]: Lowest speed is 50, reset value 42 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=58
[PicoMotor]: Lowest speed is 50, reset value 42 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=56
[PicoMotor]: Lowest speed is 50, reset value 44 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=54
[PicoMotor]: Lowest speed is 50, reset value 46 to it.
100 50
[PicoMotor]: go_turn, speed=100, diff=51
[PicoMotor]: Lowest speed is 50,

...

[PicoMotor]: go_turn, speed=-67, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 33 to it.
-67 50
[PicoMotor]: go_turn, speed=-67, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 33 to it.
-67 50
[PicoMotor]: go_turn, speed=-67, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 33 to it.
-67 50
[PicoMotor]: go_turn, speed=-66, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 34 to it.
-66 50
[PicoMotor]: go_turn, speed=-66, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 34 to it.
-66 50
[PicoMotor]: go_turn, speed=-66, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 34 to it.
-66 50
[PicoMotor]: go_turn, speed=-64, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 36 to it.
-64 50
[PicoMotor]: go_turn, speed=-64, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 36 to it.
-64 50
[PicoMotor]: go_turn, speed=-64, diff=-100
[PicoMotor]: Lowest speed is 50, reset value 36 to it.
-64 50
[PicoMotor]: go_turn, speed=-63, diff=-100
[PicoMotor]:

[PicoMotor]: go_turn, speed=-15, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -15 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=-15, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -15 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=-15, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -15 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=-10, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=-10, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=-10, diff=-100
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right

[PicoMotor]: go_turn, speed=-100, diff=-92
[PicoMotor]: Lowest speed is 50, reset value -8 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-92
[PicoMotor]: Lowest speed is 50, reset value -8 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-92
[PicoMotor]: Lowest speed is 50, reset value -8 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-90
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-90
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-90
[PicoMotor]: Lowest speed is 50, reset value -10 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-79
[PicoMotor]: Lowest speed is 50, reset value -21 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-79
[PicoMotor]: Lowest speed is 50, reset value -21 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=-79
[PicoMotor]: Lowest speed is 50, reset value -21 to it.
-100 -50
[PicoMotor]: go_turn, speed=-10

[PicoMotor]: go_turn, speed=-100, diff=94
[PicoMotor]: Lowest speed is 50, reset value -6 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=94
[PicoMotor]: Lowest speed is 50, reset value -6 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=94
[PicoMotor]: Lowest speed is 50, reset value -6 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=96
[PicoMotor]: Lowest speed is 50, reset value -4 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=96
[PicoMotor]: Lowest speed is 50, reset value -4 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=96
[PicoMotor]: Lowest speed is 50, reset value -4 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=99
[PicoMotor]: Lowest speed is 50, reset value -1 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=99
[PicoMotor]: Lowest speed is 50, reset value -1 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=99
[PicoMotor]: Lowest speed is 50, reset value -1 to it.
-100 -50
[PicoMotor]: go_turn, speed=-100, diff=100
[Pi

[PicoMotor]: go_turn, speed=-24, diff=16
[PicoMotor]: Lowest speed is 50, reset value -24 to it.
[PicoMotor]: Lowest speed is 50, reset value -34 to it.
-50 -50
[PicoMotor]: change to left_rotate
[PicoMotor]: left_rotate
[PicoMotor]: go_turn, speed=-24, diff=16
[PicoMotor]: Lowest speed is 50, reset value -24 to it.
[PicoMotor]: Lowest speed is 50, reset value -34 to it.
-50 -50
[PicoMotor]: change to left_rotate
[PicoMotor]: left_rotate
[PicoMotor]: go_turn, speed=-24, diff=16
[PicoMotor]: Lowest speed is 50, reset value -24 to it.
[PicoMotor]: Lowest speed is 50, reset value -34 to it.
-50 -50
[PicoMotor]: change to left_rotate
[PicoMotor]: left_rotate
[PicoMotor]: go_turn, speed=39, diff=16
[PicoMotor]: Lowest speed is 50, reset value 39 to it.
[PicoMotor]: Lowest speed is 50, reset value 34 to it.
50 50
[PicoMotor]: change to right_rotate
[PicoMotor]: right_rotate
[PicoMotor]: go_turn, speed=39, diff=16
[PicoMotor]: Lowest speed is 50, reset value 39 to it.
[PicoMotor]: Lowest spee

In [13]:
serial_port = I2CHelper()
# serial_port = FakeSerial()
m = PicoMotor(serial_port)

def on_value_change(v):
    speed = speed_slider.value
    diff = direct_slider.value
    
    m.go(speed, diff)

speed_slider.observe(on_value_change, names='value')
direct_slider.observe(on_value_change, names='value')

In [14]:
m.stop()